# It is a chemical plant?

Identifying images of chemical- and power plants using the fastai library.

## Setup

In [ ]:
# noqa: ruff[F405]

from pathlib import Path
from fastcore.all import *
from fastai.vision.all import *
from time import sleep
from duckduckgo_search import ddg_images
from fastdownload import download_url
import timm

In [ ]:
ROOT_PATH = Path.cwd().parent
DATA_PATH = ROOT_PATH / "data"
DATA_PATH.mkdir(exist_ok=True)

Fix VSCode Jupyter extension not displaying output of learner

In [ ]:
from IPython.display import clear_output, DisplayHandle


def update_patch(self, obj):
    clear_output(wait=True)
    self.display(obj)


DisplayHandle.update = update_patch

## Get images of chemical plants and power plants

In [ ]:
def search_images(term: str, max_images: int = 200) -> L:
    """Search images using DuckDuckGo."""
    return L(ddg_images(keywords=term, max_results=max_images)).itemgot("image")

In [ ]:
def download_and_show_image(url: str, image_name: str, dest: str):
    """Download ans show a thumbnail of an image"""
    download_url(url=url, dest=f"{dest}/{image_name}.jpg", show_progress=False)
    image = Image.open(f"{dest}/{image_name}.jpg")
    return image.to_thumb(256, 256)

In [ ]:
chemical = search_images(term="chemical plant", max_images=1)

In [ ]:
power = search_images(term="power plant", max_images=1)

In [ ]:
download_and_show_image(
    url=chemical[0], image_name="chemical", dest=f"{DATA_PATH}/chemical_or_not"
)

In [ ]:
download_and_show_image(
    url=power[0], image_name="power", dest=f"{DATA_PATH}/chemical_or_not"
)

In [ ]:
searches = ["chemical plant", "power plant"]
download_path = DATA_PATH / "chemical_or_not"

In [ ]:
for search in searches:
    dest = download_path.joinpath(search)
    dest.mkdir(exist_ok=True, parents=True)
    download_images(dest=dest, urls=search_images(term=f"{search} photo"))
    sleep(5)
    download_images(dest=dest, urls=search_images(term=f"{search} day photo"))
    sleep(5)
    download_images(dest=dest, urls=search_images(term=f"{search} night photo"))
    resize_images(
        path=download_path.joinpath(search),
        max_size=400,
        dest=download_path.joinpath(search),
    )

In [ ]:
failed_images = verify_images(get_image_files(download_path))
[Path.unlink(img) for img in failed_images]
len(failed_images)

### Train model

In [ ]:
data_loaders = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=1337),
    get_y=parent_label,
    item_tfms=[Resize(192, method="squish")],
).dataloaders(download_path)

In [ ]:
data_loaders.show_batch(max_n=6)

In [ ]:
timm.list_models()

In [ ]:
learner = vision_learner(dls=data_loaders, arch=resnet18, metrics=error_rate)
learner.fine_tune(3)

In [ ]:
learner.lr_find()

In [ ]:
learner.fine_tune(1, 0.00013182566908653826)

### Predict previously downloaded images

In [ ]:
is_chemical, _, probs = learner.predict(
    PILImage.create(f"{DATA_PATH}/chemical_or_not/chemical.jpg")
)

In [ ]:
print(f"The image above is a {is_chemical}", f"I am {probs[0]:.2f} sure!", sep="\n")

### Export model

In [ ]:
learner.export(Path("models/chemical.pkl"))